In [ ]:
# Modelo YOLO Version 3 para detección de objetos
# Ruta al repositorio 
# C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/
# Ruta al fichero de configuracion yolov3.cfg
#C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/config
# Ruta a los pesos prenetrandos yolov3.weights
# C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/weights/

# Importamos librerias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import sys
import os
import numpy as np 
import pandas as pd

import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from tqdm import tqdm 

import cv2
print("Liberias importadas correctamente")

# Configuración de rutas
# Ruta donde hemos clonado el repositorio de Erik Lindernoren.
YOLOV3_REPO_PATH = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/'
YOLOV3_MODELS_PATH = os.path.join(YOLOV3_REPO_PATH, 'pytorchyolo')
print(f"Ruta del repositorio YOLOv3: {YOLOV3_REPO_PATH}")
print(f"Ruta de los modelos YOLOv3: {YOLOV3_MODELS_PATH}")

# Rutas de Archivos Específicos
# Archivo de configuracion yolov3.cfg
CONFIG_PATH = os.path.join(YOLOV3_REPO_PATH, 'config', 'yolov3.cfg')
CONFIG_PATH = CONFIG_PATH.replace('\\', '/')  # Asegúrate de usar barras normales para evitar problemas en Linux/Mac
print(f"Ruta del archivo de configuración YOLOv3: {CONFIG_PATH}")

# Archivo de pesos .weights descargado de https://github.com/patrick013/Object-Detection---Yolov3.git
WEIGHTS_PATH = os.path.join(YOLOV3_REPO_PATH, 'yolov3.weights')
WEIGHTS_PATH = WEIGHTS_PATH.replace('\\', '/')  # Asegúrate de usar barras normales para evitar problemas en Linux/Mac
print(f"Ruta del archivo de pesos YOLOv3: {WEIGHTS_PATH}")

# Añadimos esta ruta al PYTHONPATH para que Python pueda encontrar los módulos.
sys.path.append(YOLOV3_REPO_PATH)
sys.path.append(YOLOV3_MODELS_PATH)
print(f"Rutas añadidas al PYTHONPATH: {YOLOV3_REPO_PATH} y {YOLOV3_MODELS_PATH}")

# Importamos las clases necesarias del repositorio.
# Darknet y YOLOLayer son las clases principales del modelo.
from models import Darknet, YOLOLayer 

# Detección del Dispositivo (CPU o GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Trabajando en el dispositivo: {device}")

# DEFINICIONES DE CLASES Y FUNCIONES

# Definicion de la función intersection_over_union
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    return iou

# Definición de la clase YOLOv3Loss
class YOLOv3Loss(nn.Module):
        return total_loss, {"box_loss": box_loss, "obj_loss": obj_loss, "noobj_loss": noobj_loss, "class_loss": class_loss}

# Definición de la clase BloodCellDataset
class BloodCellDataset(Dataset):
        return image, yolo_bboxes

# INICIO DE LA CONFIGURACIÓN DEL MODELO YOLOv3

# PASO 1: Instanciar el Modelo YOLOv3 (para 80 clases, usando el .cfg original)
# La clase Darknet de Erik Lindernoren construye el modelo leyendo el archivo yolov3.cfg.
# Esto crea el modelo con la arquitectura esperada por el archivo yolov3.weights.
print(f"Cargando la arquitectura del modelo desde: {CONFIG_PATH} (con classes=80)")
model = Darknet(CONFIG_PATH)
model.to(device) # Mueve el modelo al dispositivo (GPU/CPU)
print("Modelo YOLOv3 cargado correctamente en el dispositivo: ", device)

# PASO 2: Cargamos los Pesos Pre-entrenados
# El método load_darknet_weights() es el encargado de leer el archivo yolov3.weights.
try:
    print(f"Intentando cargar pesos pre-entrenados desde: {WEIGHTS_PATH}")
    model.load_darknet_weights(WEIGHTS_PATH)
    print("Pesos pre-entrenados cargados con éxito.")

except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo de pesos en {WEIGHTS_PATH}.")
    print("El modelo se inicializará con pesos aleatorios (NO se usará transfer learning).")
    print("¡ADVERTENCIA! Entrenar desde cero con solo 300 imágenes será extremadamente difícil.")
except Exception as e:
    print(f"ERROR al cargar los pesos pre-entrenados: {e}")
    print("El modelo se inicializará con pesos aleatorios (NO se usará transfer learning).")
    print("¡ADVERTENCIA! Entrenar desde cero con solo 300 imágenes será extremadamente difícil.")

# PASO 3: Adapatacion del modelo para las 3 clases (FINE-TUNING EN MEMORIA)
# Esto debemos hacerlo DESPUÉS de haber cargado los pesos del modelo de 80 clases.

print("\nAdaptando las capas de predicción a 3 clases...")

yolo_layer_index_in_model_yolo_layers = 0 # Para asignar los nuevos YOLOLayer a la lista correcta

for i, module_def in enumerate(model.module_defs):
    if module_def["type"] == "yolo":
        # i es el índice de la capa YOLOLayer en model.module_defs y module_list
        
        # 1. Reemplazar la capa Conv2d de predicción final
        pred_conv_sequential_idx = i - 1 
        pred_conv_layer_old = model.module_list[pred_conv_sequential_idx][0] 
        
        yolo_layer_old_instance = model.yolo_layers[yolo_layer_index_in_model_yolo_layers]
        
        new_out_channels = len(yolo_layer_old_instance.anchors) * (5 + NUM_CLASSES_YOUR_DATASET)
        
        new_pred_conv_layer = nn.Conv2d(pred_conv_layer_old.in_channels, new_out_channels, 
                                        kernel_size=pred_conv_layer_old.kernel_size,
                                        stride=pred_conv_layer_old.stride,
                                        padding=pred_conv_layer_old.padding,
                                        bias=True 
                                        )
        model.module_list[pred_conv_sequential_idx] = nn.Sequential(new_pred_conv_layer)
        
        # 2. Reemplazar la instancia de YOLOLayer en model.module_list y model.yolo_layers
        # ¡CORREGIDO! La YOLOLayer está en module_list[i]
        
        # Obtenemos los anchors y stride de la instancia antigua para la nueva YOLOLayer
        # Esto es correcto ya que estos atributos sí existen en yolo_layer_old_instance
        anchors_for_new_layer = yolo_layer_old_instance.anchors.tolist()
        stride_for_new_layer = yolo_layer_old_instance.stride
        
        # Creamos una NUEVA instancia de YOLOLayer
        new_yolo_layer = YOLOLayer(anchors_for_new_layer, NUM_CLASSES_YOUR_DATASET, new_coords=False)
        
        # Sustituimos la YOLOLayer antigua en el `module_list` del modelo
        # Esto es crucial porque el forward de Darknet itera sobre module_list
        model.module_list[i] = nn.Sequential(new_yolo_layer) # Reemplaza el Sequential que contiene la YOLOLayer antigua
        
        # También actualizamos la referencia en `model.yolo_layers`
        model.yolo_layers[yolo_layer_index_in_model_yolo_layers] = new_yolo_layer
        
        yolo_layer_index_in_model_yolo_layers += 1 

print("Capas YOLOLayer y sus capas de predicción Conv2d adaptadas para 3 clases.")

# PASO 4: Congelamos las capas para Fine-Tuning
# Es CRUCIAL congelar la mayoría de las capas del backbone (Darknet-53)
# y dejar entrenables las capas del head (las que predicen las cajas).
# Esto evita que el modelo "olvide" lo que aprendió en el dataset grande.

print("\nConfigurando capas para Fine-Tuning:")
# Iteramos a través de los módulos y parámetros del modelo.
# Las primeras ~74-75 capas de su `module_list` corresponden al backbone (Darknet-53 puro).
# Las capas posteriores (más de 75) son parte del head de YOLOv3 y deben ser entrenables.
# Las capas YOLOLayer en sí no tienen parámetros entrenables, pero sus capas `conv` previas sí.
for i, (name, param) in enumerate(model.named_parameters()):
    if i < 75:  # Índices de las capas del backbone (heurístico, basado en estructura Darknet-53)
        param.requires_grad = False  # Congelar
    else:
        param.requires_grad = True   # Descongelar (para el head)
    
    # Línea para depuración: puedes descomentarla para ver el estado de cada capa
    # print(f"  Capa: {name}, Entrenable: {param.requires_grad}")

# --- Verificación de Capas Entrenables ---
print("\nVerificación de capas que se entrenarán ('requires_grad=True'):")
trainable_params_count = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)
        trainable_params_count += param.numel()

total_params = sum(p.numel() for p in model.parameters())
print(f"\nTotal de parámetros entrenables: {trainable_params_count / 1e6:.2f} M")
print(f"Total de parámetros congelados: {(total_params - trainable_params_count) / 1e6:.2f} M")
print(f"Total de parámetros en el modelo: {total_params / 1e6:.2f} M")

# PASO 5: Prueba Final de la Pasada hacia Adelante (sanity check)
# En modo eval(), YOLOLayer devuelve predicciones decodificadas y aplanadas.
print("\nRealizando una pasada hacia adelante para verificar la configuración del modelo...")
# Modo EVAL: No se entrena, solo se evalúa la salida del modelo.

IMG_SIZE = 416  # Tamaño de imagen esperado por el modelo (debe coincidir con el tamaño de entrada del modelo)
NUM_CLASSES_YOUR_DATASET = 3  # Número de clases en tu dataset (BCCD: RBC, WBC, Plaquetas)
    
model.eval() 
dummy_input = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(device) 
with torch.no_grad():
    predictions = model(dummy_input)

print(f"\nShape de la salida del modelo después de cargar pesos y adaptar a {NUM_CLASSES_YOUR_DATASET} clases (en modo EVAL):")
# Ajustamos las expectativas de forma para reflejar la salida aplanada y decodificada
print(f"  Escala 13x13: {predictions[0].shape} (Esperado: [N, 3*13*13, 5+C])") 
print(f"  Escala 26x26: {predictions[1].shape} (Esperado: [N, 3*26*26, 5+C])")
print(f"  Escala 52x52: {predictions[2].shape} (Esperado: [N, 3*52*52, 5+C])")

# No necesitamos los asserts basados en el formato crudo aquí, ya que el formato de evaluación es diferente
# Los asserts que teníamos antes son para el formato crudo (en modo train)
# expected_output_channels = 3 * (5 + NUM_CLASSES_YOUR_DATASET)
# assert predictions[0].shape[1] == expected_output_channels # Esto no es verdad en modo eval()
# assert predictions[1].shape[1] == expected_output_channels # Esto no es verdad en modo eval()
# assert predictions[2].shape[1] == expected_output_channels # Esto no es verdad en modo eval()
print(f"¡Las dimensiones de salida para {NUM_CLASSES_YOUR_DATASET} clases son correctas en modo EVAL!")

print("\n--- ¡Fase de Configuración del Modelo YOLOv3 Completada Exitosamente! ---")

# INICIO DE LA CONFIGURACIÓN DEL ENTRENAMIENTO
# Preparando el entrenamiento de YOLOv3

# Hiperparámetros de training
LEARNING_RATE = 1e-4
NUM_EPOCHS = 50 
WEIGHT_DECAY = 1e-5
GRADIENT_CLIP_VALUE = 0.1
BATCH_SIZE = 2

# Parámetros Generales
# Número de clases del dataset BCCD (Glóbulos Rojos, Glóbulos Blancos, Plaquetas).
NUM_CLASSES = 3 
# Tamaño de la imagen de entrada para el modelo YOLOv3 (típicamente 416x416 o 608x608).
IMG_SIZE = (416, 416)

# Anchor Boxes de YOLOv3 calculadas con K-means para el dataset de BCCD
ANCHORS = [
    [(227, 210), (179, 155), (124, 111)],  # Anchors para la escala más grande ... Escala 0 (grid 13x13) (stride 32, detecta objetos grandes)
    [(105, 113), (104, 96), (80, 109)],    # Anchors para la escala media ... Escala 1 (grid 26X26) (stride 16, detecta objetos medianos)
    [(112, 75), (87, 82), (39, 38)]        # Anchors para la escala más pequeña ... Escala 2 (grid 52x52) (stride 8, detecta objetos pequeños)
]

# Definición de las transformaciones de Albumentations
# Define el tamaño de entrada de tu modelo YOLOv3 (416x416)
YOLO_INPUT_SIZE = (416, 416) 

# Las transformaciones para el set de training incluyen aumentacion de COLOR/APARIENCIA
train_transforms = A.Compose([
    # Redimensionamiento y Relleno
    A.LongestMaxSize(max_size=YOLO_INPUT_SIZE[0], p=1.0), 
    A.PadIfNeeded(min_height=YOLO_INPUT_SIZE[0], min_width=YOLO_INPUT_SIZE[1], border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
    
    # Transformaciones de Color y Apariencia
    A.RGBShift(r_shift_limit=10, g_shift_limit=10, b_shift_limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(p=0.2),
    A.Blur(blur_limit=3, p=0.1), # Asegúrate de que blur_limit es impar y no demasiado grande
    
    # Normalización y Conversión a Tensor
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2(), 
], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])) # El formato 'albumentations' espera y devuelve normalizado [0,1]

# Las transformaciones de validación/prueba se mantienen minimalistas
val_test_transforms = A.Compose([
    A.LongestMaxSize(max_size=YOLO_INPUT_SIZE[0], p=1.0), 
    A.PadIfNeeded(min_height=YOLO_INPUT_SIZE[0], min_width=YOLO_INPUT_SIZE[1], border_mode=cv2.BORDER_CONSTANT, value=0, p=1.0),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
], bbox_params=A.BboxParams(format='albumentations', label_fields=['class_labels'])) 

# Definición de la función Collate_fn para el DataLoader
def collate_fn(batch):
    images = []
    bboxes = []
    for img, bbox_target in batch:
        images.append(img)
        bboxes.append(bbox_target) 
    images = torch.stack(images, 0)
    return images, bboxes

# Definición de la Lógica de División del Dataset y Creación de DataLoaders
# Modificada para que visualice las imágenes con las bounding boxes

if __name__ == '__main__':
    # RUTAS A LOS DATOS
    DATA_ROOT = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset'
    CSV_FILE = os.path.join(DATA_ROOT, 'annotations.csv') 
        
    # Parámetros de la división
    TEST_SPLIT_RATIO = 0.15    
    VAL_SPLIT_RATIO = 0.15     
    RANDOM_SEED = 42           

    BATCH_SIZE = 8
    NUM_WORKERS = 0 # Deja en 0 para depuración, luego puedes aumentarlo a 4-8

    # --- Cargar todas las anotaciones y obtener nombres de archivo únicos ---
    print(f"Cargando todas las anotaciones desde: {CSV_FILE}")
    full_df = pd.read_csv(CSV_FILE)
    
    # Obtener la lista de nombres de archivo únicos presentes en el CSV
    all_image_filenames = full_df['filename'].unique().tolist()
    print(f"Total de {len(all_image_filenames)} imágenes únicas encontradas en el CSV.")

    # --- Dividir los nombres de archivo en entrenamiento y test ---
    train_val_filenames, test_filenames = train_test_split(
        all_image_filenames, 
        test_size=TEST_SPLIT_RATIO, 
        random_state=RANDOM_SEED
    )
    
    train_filenames, val_filenames = train_test_split(
        train_val_filenames, 
        test_size=VAL_SPLIT_RATIO / (1 - TEST_SPLIT_RATIO), 
        random_state=RANDOM_SEED
    )

    print(f"Imágenes para entrenamiento: {len(train_filenames)}")
    print(f"Imágenes para validación: {len(val_filenames)}")
    print(f"Imágenes para prueba: {len(test_filenames)}")

    # --- Crear DataFrames de anotaciones para cada split ---
    train_df = full_df[full_df['filename'].isin(train_filenames)].copy()
    val_df = full_df[full_df['filename'].isin(val_filenames)].copy()
    test_df = full_df[full_df['filename'].isin(test_filenames)].copy()

    # --- Crear instancias del Dataset y DataLoader para cada split ---
    train_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=train_df, 
        image_size=YOLO_INPUT_SIZE,
        transform=train_transforms
    )
    val_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=val_df, 
        image_size=YOLO_INPUT_SIZE,
        transform=val_test_transforms 
    )
    test_dataset = BloodCellDataset(
        data_root=DATA_ROOT,
        annotations_df=test_df, 
        image_size=YOLO_INPUT_SIZE,
        transform=val_test_transforms 
    )

    train_dataloader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )
    val_dataloader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )
    test_dataloader = DataLoader(
        test_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=NUM_WORKERS, collate_fn=collate_fn, pin_memory=True
    )

    print("\nDataset y DataLoaders de entrenamiento, validación y prueba configurados exitosamente.")

    # --- Verificación de la carga de un lote de entrenamiento ---
    print("\nVerificando la carga de un lote de entrenamiento...")
    MAX_BATCHES_TO_CHECK = 10 
    found_image_with_boxes = False

    for batch_idx, (images, targets) in enumerate(train_dataloader):
        print(f"Tamaño del lote {batch_idx+1}: Imágenes: {images.shape}, Targets: {len(targets)}")
        
        # Buscar una imagen con cajas en el lote actual
        for img_idx in range(len(targets)):
            if targets[img_idx].numel() > 0: 
                print(f"--- Encontrada imagen con {targets[img_idx].shape[0]} cajas en el lote {batch_idx+1}, imagen {img_idx+1} ---")
                print(f"Ejemplo de target para esta imagen (clase, cx, cy, w, h normalizados):")
                print(targets[img_idx][0])
                
                # --- Lógica de visualización ---
                mean = torch.tensor((0.485, 0.456, 0.406)).view(3, 1, 1).to(images[img_idx].device)
                std = torch.tensor((0.229, 0.224, 0.225)).view(3, 1, 1).to(images[img_idx].device)
                
                img_display_rgb = (images[img_idx] * std + mean) * 255
                img_display_rgb = img_display_rgb.permute(1, 2, 0).cpu().numpy().astype(np.uint8)
                img_display_bgr = cv2.cvtColor(img_display_rgb, cv2.COLOR_RGB2BGR)
                
                img_h, img_w = img_display_bgr.shape[:2]
                
                CLASS_ID_TO_NAME_MAP = {0: 'RBC', 1: 'WBC', 2: 'Platelets'}
                CLASS_COLORS_MAP = {0: (0, 0, 255), 1: (0, 255, 0), 2: (255, 0, 0)} # BGR

                print("\nVisualizando la imagen con GT Boxes (presiona cualquier tecla para cerrar)...")
                for bbox_yolo in targets[img_idx].tolist():
                    class_id, cx, cy, w, h = bbox_yolo
                    
                    x_min_norm = cx - w/2
                    y_min_norm = cy - h/2
                    x_max_norm = cx + w/2
                    y_max_norm = cy + h/2

                    x_min_px = int(x_min_norm * img_w)
                    y_min_px = int(y_min_norm * img_h)
                    x_max_px = int(x_max_norm * img_w)
                    y_max_px = int(y_max_norm * img_h)

                    color = CLASS_COLORS_MAP.get(int(class_id), (255, 255, 255)) 
                    cv2.rectangle(img_display_bgr, (x_min_px, y_min_px), (x_max_px, y_max_px), color, 2)

                    label_text = f"{CLASS_ID_TO_NAME_MAP.get(int(class_id), 'Unknown')}"
                    text_size = cv2.getTextSize(label_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)[0]
                    text_x = x_min_px
                    text_y = y_min_px - 5 if y_min_px - 5 > 5 else y_min_px + text_size[1] + 5
                    
                    cv2.rectangle(img_display_bgr, (text_x, text_y - text_size[1] - 5), 
                                (text_x + text_size[0] + 5, text_y + 5), color, -1)
                    cv2.putText(img_display_bgr, label_text, (text_x, text_y), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

                cv2.imshow("Imagen con GT Boxes", img_display_bgr)
                cv2.waitKey(0) 
                cv2.destroyAllWindows()
                
                found_image_with_boxes = True
                break 
        
        if found_image_with_boxes or batch_idx + 1 >= MAX_BATCHES_TO_CHECK:
            break 

    if not found_image_with_boxes:
        print(f"\nNo se encontró ninguna imagen con bounding boxes en los primeros {MAX_BATCHES_TO_CHECK} lotes.")
        print("Esto podría deberse a que todas las imágenes mostradas no tenían bboxes o fueron filtradas.")
        print("Considera revisar:")
        print("1. El contenido de 'annotations.csv' para asegurar que hay bboxes válidas.")
        print("2. Los filtros en BloodCellDataset (xmin >= xmax, etc.).")
        print("3. Los parámetros de bbox en Albumentations (min_area, min_visibility).")
        print("4. Si RandomCrop está eliminando demasiadas bboxes si son pequeñas o están en los bordes.")

# Bucle de Entrenamiento Principal

if __name__ == "__main__":
    # Listas para almacenar el historial de pérdidas
    train_losses = []
    val_losses = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando dispositivo: {device}")

    model = DummyYOLOv3Model(num_classes=NUM_CLASSES).to(device)
    
    loss_fn = YOLOv3Loss(
        anchors=ANCHORS, 
        num_classes=NUM_CLASSES,
        img_size=IMG_SIZE,
        lambda_coord=1.0, lambda_noobj=1.0, lambda_obj=1.0, lambda_class=1.0, 
        ignore_iou_threshold=0.5
    ).to(device)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

    print("\n--- Iniciando el Bucle de Entrenamiento Principal ---")

    for epoch in range(NUM_EPOCHS):
        model.train() # Modo entrenamiento
        total_train_loss = 0.0
        
        loop = tqdm(train_dataloader, leave=True, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} (Train)")
        
        for batch_idx, (images, targets) in enumerate(loop):
            images = images.to(device)
            # targets ahora es un tensor, así que también lo movemos al dispositivo
            targets = targets.to(device) # <--- CORRECCIÓN: Mover targets al dispositivo
            
            outputs = model(images)
            loss, _ = loss_fn(outputs, targets) 
            
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP_VALUE) 
            optimizer.step()
            
            total_train_loss += loss.item()
            loop.set_postfix(loss=total_train_loss/(batch_idx+1)) 
            
        scheduler.step() 
        avg_train_loss = total_train_loss / len(train_dataloader)
        train_losses.append(avg_train_loss) 
        
        print(f"Época {epoch+1} - Tasa de Aprendizaje: {optimizer.param_groups[0]['lr']:.6f}")
        print(f"Época {epoch+1} - Pérdida de Entrenamiento Promedio: {avg_train_loss:.4f}")

        # --- Fase de Validación ---
        model.eval() # Modo evaluación
        total_val_loss = 0.0
        with torch.no_grad(): 
            for batch_idx_val, (images_val, targets_val) in enumerate(val_dataloader):
                images_val = images_val.to(device)
                # targets_val también debe moverse al dispositivo
                targets_val = targets_val.to(device) # <--- CORRECCIÓN: Mover targets_val al dispositivo
                
                outputs_val = model(images_val)
                val_loss, _ = loss_fn(outputs_val, targets_val)
                total_val_loss += val_loss.item()
        
        avg_val_loss = total_val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss) 
        
        print(f"Época {epoch+1} - Pérdida de Validación Promedio: {avg_val_loss:.4f}")

    print("\n--- Entrenamiento Finalizado ---")
    print("El modelo ha completado el entrenamiento simulado.")
    print("Recuerda: Los valores de pérdida son para un modelo DUMMY con predicciones aleatorias.")
    print("Para un entrenamiento real, reemplaza el DummyYOLOv3Model con tu modelo real.")

    print("\nHistorial de Pérdidas de Entrenamiento por Época:", train_losses)
    print("Historial de Pérdidas de Validación por Época:", val_losses)


Liberias importadas correctamente
Ruta del repositorio YOLOv3: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/
Ruta de los modelos YOLOv3: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/pytorchyolo
Ruta del archivo de configuración YOLOv3: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/config/yolov3.cfg
Ruta del archivo de pesos YOLOv3: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/yolov3.weights
Rutas añadidas al PYTHONPATH: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/ y C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/pytorchyolo
Trabajando en el dispositivo: cpu
Cargando todas las anotaciones desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset\annotations.csv
Total de 364 imágenes únicas encontradas en el CSV.
Imágenes para entrenamiento: 254
Imágenes para validación: 55
Imágenes para prueba: 55
Dataset inicializado con 254 imágenes.
Dataset inicializado con 55 imágenes.
Dataset inicializ

Epoch 1/50 (Train):   0%|          | 0/32 [00:00<?, ?it/s]

DEBUG: Imagen original (H, W): (480, 640)
DEBUG: __getitem__ para BloodImage_00356.jpg. Bboxes iniciales (píxeles): 17
DEBUG: Primer bbox pixel: [279, 310, 396, 422, 0]
DEBUG: Bboxes normalizadas (iniciales): 17
DEBUG: Primer bbox normalizada (inicial): [0.4359375, 0.6458333333333334, 0.61875, 0.8791666666666667], clase: 0
DEBUG: Bboxes después de Albumentations (raw, deberían estar normalizadas): 17
DEBUG: Primer bbox después de Albumentations (raw, deberían estar normalizadas): (0.4359375, 0.609375, 0.6187500000000001, 0.784375)
DEBUG: Bboxes finales antes de YOLO format: 17
DEBUG: Primer bbox final antes de YOLO format: (0.4359375, 0.609375, 0.6187500000000001, 0.784375)
DEBUG: Bboxes finales en formato YOLO: 17
DEBUG: Primer bbox YOLO: [0, 0.52734375, 0.696875, 0.18281250000000016, 0.17500000000000004]
DEBUG: Imagen original (H, W): (480, 640)
DEBUG: __getitem__ para BloodImage_00313.jpg. Bboxes iniciales (píxeles): 12
DEBUG: Primer bbox pixel: [24, 94, 131, 202, 0]
DEBUG: Bboxes n

AttributeError: 'list' object has no attribute 'to'